# Cuaderno 33: Problema de corte de material (Cutting Stock Problem) 

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\S}{\mathcal{S}}$


Dados: 
* un conjunto de rollos, cada uno de longitud $L$; y,
* $m$ órdenes de corte, cada una de las cuales consiste en cortar de los rollos disponibles $b_i$ items de longitud $w_i$, con $w_i < L$, $i \in \{1, \ldots, m\}$.

El problema del corte de material (*Cutting Stock Problem*) consiste en determinar la manera de satisfacer todas las órdenes de corte empleando la menor cantidad posible de rollos. 

Para formular este problema como un programa lineal entero, determinaremos primero una cota superior $N$ al número de rollos necesarios para cumplir con todas las órdenes. Notar que si cortamos cada item de un rollo distinto, necesitamos como máximo $\sum_{i=1}^m b_i$ rollos. Otra cota más ajustada consiste en determinar la cantidad de rollos necesaria para satisfacer cada pedido de manera independiente. Se llega así al valor $N:= \sum_{i=1}^m \lceil \frac{b_i w_i}{L - w_i} \rceil$.

Definimos ahora dos conjuntos de variables de decisión: las variables binarias $y_j$, con $j \in \{1, \ldots, N\}$,
indican si se usa o no el rollo $j$ en la solución. Las variables enteras $x_{ij}$, por su parte, indican la cantidad de items de la orden $i$ que serán cortados del rollo $j$. Con estas variables, el problema de corte de material puede formularse como el siguiente programa lineal entero:
\begin{align*}
\min &\sum_{j \in J} y_j\\ 
& \mbox{s.r.}\\
&\sum_{j \in J} x_{ij} \geq b_i, \quad \forall i \in I,\\
&\sum_{i \in I} w_i x_{ij} \leq L y_j, \quad \forall j \in J,\\
& x_{ij} \in \ZZ, \quad \forall i \in I, j \in J,\\
& y_j \in \{0, 1\}, \forall j \in J,
\end{align*}
donde $I:= \{1, \ldots, m \}$ es el conjunto de órdenes de corte y $J:= \{1, \ldots, N \}$ es el conjunto de rollos disponibles.


Implementaremos a continuación este modelo utilizando la interfaz Python del solver Gurobi.


Definimos primero los datos. Generaremos $m$ órdenes de corte, consistentes en cortar items de longitud $1, 3, \ldots, 2m+1$. La cantidad de items a cortar de cada tipo estará dada por un número aleatorio entre 1 y 50, obtenido llamando a la función `randint` del módulo `random`. La longitud de los rollos será $L := 3m$.

In [22]:
from gurobipy import *
import random as rm
import math

# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de ordenes
m = 50

# Conjunto de indices para las ordenes, I:={1,...,m}
I = range(1, m+1)

# Diccionarios con las longitudes y cantidades demandadas de cada item
w = {} # longitudes 1, 3, ... , 2m+1
b = {} # demandas (aleatorio entre 1 y 50)
for i in I:
    w[i]= rm.randint(m, 4*m)
    b[i]= rm.randint(10, 20)

L = 5*m

# Cantidad maxima de rollos a utilizar
N = sum([int(math.ceil(w[i]*b[i]/(L - w[i]))) for i in I])

# Conjuntos de indices de rollos
J = range(1, N+1)

print w
print b
print N

{1: 177, 2: 113, 3: 127, 4: 168, 5: 121, 6: 187, 7: 92, 8: 143, 9: 187, 10: 172, 11: 96, 12: 185, 13: 121, 14: 115, 15: 187, 16: 122, 17: 89, 18: 132, 19: 158, 20: 174, 21: 50, 22: 181, 23: 99, 24: 78, 25: 86, 26: 171, 27: 62, 28: 126, 29: 66, 30: 156, 31: 172, 32: 195, 33: 138, 34: 140, 35: 136, 36: 78, 37: 142, 38: 121, 39: 164, 40: 189, 41: 185, 42: 131, 43: 156, 44: 172, 45: 185, 46: 193, 47: 118, 48: 200, 49: 169, 50: 142}
{1: 18, 2: 12, 3: 14, 4: 13, 5: 16, 6: 15, 7: 18, 8: 12, 9: 20, 10: 19, 11: 18, 12: 17, 13: 11, 14: 16, 15: 20, 16: 19, 17: 18, 18: 10, 19: 14, 20: 17, 21: 15, 22: 12, 23: 19, 24: 16, 25: 20, 26: 14, 27: 13, 28: 20, 29: 16, 30: 16, 31: 15, 32: 16, 33: 14, 34: 14, 35: 13, 36: 12, 37: 17, 38: 10, 39: 19, 40: 19, 41: 20, 42: 14, 43: 13, 44: 19, 45: 16, 46: 16, 47: 17, 48: 20, 49: 10, 50: 15}
1300


Definimos ahora el modelo , sus variables y la función objetivo. 


In [3]:
# Crear el objeto modelo
m = Model('cutting-stock')

# Crear las variables asociadas a cantidad de items cortados de cada rollo
x = m.addVars(I, J, name="x", vtype=GRB.INTEGER)

# Crear las variables asociadas al uso de rollos
y = m.addVars(J, name="y", vtype=GRB.BINARY)

# Crear la funcion objetivo
m.setObjective(y.sum('*'), GRB.MINIMIZE)


Academic license - for non-commercial use only


Añadimos las restricciones de satisfacción de la demanda. 

In [5]:
# Restricciones de demanda
m.addConstrs((x.sum(i,'*') >= b[i] for i in I), "demanda");


Añadimos las restricciones de enforzamiento que impiden asignar items a rollos sin utilizar, y garantizan además que la longitud de los rollos sea respetada al cortar los items:

In [7]:
# Restricciones de tamanio de rollos
m.addConstrs((sum([w[i]*x[i,j] for i in I]) <= L*y[j] for j in J), "tam_rollo");


Finalmente, resolvemos el modelo. Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [8]:
# Terminar al alcanzar un Gap del 10%
m.Params.MIPGap = 0.1

# Terminar luego de 240 segundos
m.Params.TimeLimit = 240

# Activar nuevamente los mensajes de salida
m.Params.OutputFlag = 1

# Resolver el problema principal
m.optimize()

Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter TimeLimit to 240.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 544 rows, 5502 columns and 16244 nonzeros
Variable types: 0 continuous, 5502 integer (262 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 5e+01]
Presolve removed 262 rows and 0 columns
Presolve time: 0.12s
Presolved: 282 rows, 5502 columns, 10742 nonzeros
Variable types: 0 continuous, 5502 integer (1834 binary)
Found heuristic solution: objective 262.0000000
Found heuristic solution: objective 246.0000000

Root relaxation: objective 2.442833e+02, 877 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incum

Escribir la solución óptima:

In [12]:
# Recuperar variables y_j cuyo valor sea 1:
for j in J:
    if y[j].getAttr('X') > 0.1:
        # Recuperar valores x_ij asociados al rollo j y ponernlos en una lista
        L =[int(x[i,j].getAttr('X')) for i in I]
        # Transformar la lista en una cadena de caracteres e imprimirla
        S = ['{:2} '.format(k) for k in L]
        print ('[{}]'.format(''.join(S)))


[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1 ]
[ 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 

## Código completo

Se reproduce a continuación el código completo del modelo anterior. Notar que no es necesario generar la primera columna "por separado" como lo hicimos en este ejemplo, con el propósito de explicar mejor el algoritmo.

In [24]:
# Implementación de modelos lineales enteros
# Problema del corte de material (Cutting-Stock)

# Luis M. Torres (EPN 2019)

from gurobipy import *
import random as rm
import math

# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de ordenes
m = 50

# Conjunto de indices para las ordenes, I:={1,...,m}
I = range(1, m+1)

# Diccionarios con las longitudes y cantidades demandadas de cada item
w = {} # longitudes 1, 3, ... , 2m+1
b = {} # demandas (aleatorio entre 1 y 50)
for i in I:
    w[i]= rm.randint(m, 4*m)
    b[i]= rm.randint(10, 20)

L = 5*m

# Cantidad maxima de rollos a utilizar
N = sum([int(math.ceil(w[i]*b[i]/(L - w[i]))) for i in I])

# Conjuntos de indices de rollos
J = range(1, N+1)

print w
print b
print N

try:
    # Crear el objeto modelo
    m = Model('cutting-stock')

    # Crear las variables asociadas a cantidad de items cortados de cada rollo
    x = m.addVars(I, J, name="x", vtype=GRB.INTEGER)

    # Crear las variables asociadas al uso de rollos
    y = m.addVars(J, name="y", vtype=GRB.BINARY)

    # Crear la funcion objetivo
    m.setObjective(y.sum('*'), GRB.MINIMIZE)

    # Restricciones de demanda
    m.addConstrs((x.sum(i,'*') >= b[i] for i in I), "demanda");

    # Restricciones de tamanio de rollos
    m.addConstrs((sum([w[i]*x[i,j] for i in I]) <= L*y[j] for j in J), "tam_rollo");

    # Escribir el modelo PM inicial a un archivo
    # m.write('cutting-stock-inicial.lp')
    

    # Terminar al alcanzar un Gap del 10%
    m.Params.MIPGap = 0.1

    # Terminar luego de 240 segundos
    m.Params.TimeLimit = 240

    # Activar nuevamente los mensajes de salida
    m.Params.OutputFlag = 1

    # Resolver el problema principal
    m.optimize()

    # Escribir la solucion
    # Recuperar variables y_j cuyo valor sea 1:
    # for j in J:
    #    if y[j].getAttr('X') > 0.1:
            # Recuperar valores x_ij asociados al rollo j y ponernlos en una lista
    #        L =[int(x[i,j].getAttr('X')) for i in I]
            # Transformar la lista en una cadena de caracteres e imprimirla
    #        S = ['{:2} '.format(k) for k in L]
    #        print ('[{}]'.format(''.join(S)))
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

{1: 177, 2: 113, 3: 127, 4: 168, 5: 121, 6: 187, 7: 92, 8: 143, 9: 187, 10: 172, 11: 96, 12: 185, 13: 121, 14: 115, 15: 187, 16: 122, 17: 89, 18: 132, 19: 158, 20: 174, 21: 50, 22: 181, 23: 99, 24: 78, 25: 86, 26: 171, 27: 62, 28: 126, 29: 66, 30: 156, 31: 172, 32: 195, 33: 138, 34: 140, 35: 136, 36: 78, 37: 142, 38: 121, 39: 164, 40: 189, 41: 185, 42: 131, 43: 156, 44: 172, 45: 185, 46: 193, 47: 118, 48: 200, 49: 169, 50: 142}
{1: 18, 2: 12, 3: 14, 4: 13, 5: 16, 6: 15, 7: 18, 8: 12, 9: 20, 10: 19, 11: 18, 12: 17, 13: 11, 14: 16, 15: 20, 16: 19, 17: 18, 18: 10, 19: 14, 20: 17, 21: 15, 22: 12, 23: 19, 24: 16, 25: 20, 26: 14, 27: 13, 28: 20, 29: 16, 30: 16, 31: 15, 32: 16, 33: 14, 34: 14, 35: 13, 36: 12, 37: 17, 38: 10, 39: 19, 40: 19, 41: 20, 42: 14, 43: 13, 44: 19, 45: 16, 46: 16, 47: 17, 48: 20, 49: 10, 50: 15}
1300
Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter TimeLimit to 240.0
   Prev: 1e+100  Min: 0.0  M